In [1]:
import requests, zipfile, io
import pandas as pd

# Récupération des données socio-économiques 

Disponibles sur le site de l'INSEE

In [2]:
url = 'https://www.insee.fr/fr/statistiques/fichier/5359146/dossier_complet.zip'

In [3]:
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("INSEE/")

# Récupération des données des élections municipales de 2014

Disponibles sur le site regard citoyen. Méthode choisie à cause de problèmes majeurs d'encodage.

In [107]:
!curl -O https://data.regardscitoyens.org/elections/2014_municipales/municipales-2014-résultats-bureaux_vote-tour1.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 35.2M  100 35.2M    0     0  14.8M      0  0:00:02  0:00:02 --:--:-- 14.8M


In [108]:
municipales_t1 = pd.read_csv(filepath_or_buffer='municipales-2014-résultats-bureaux_vote-tour1.csv', encoding='l5', sep=';', error_bad_lines=False)

/Users/henri/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/henri/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [129]:
!curl -O https://data.regardscitoyens.org/elections/2014_municipales/municipales-2014-résultats-bureaux_vote-tour2.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 7213k  100 7213k    0     0  13.4M      0 --:--:-- --:--:-- --:--:-- 13.4M


In [198]:
municipales_t2 = pd.read_csv(filepath_or_buffer='municipales-2014-résultats-bureaux_vote-tour2.csv', encoding='l5', sep=';', error_bad_lines=False)

/Users/henri/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


# Nettoyage et élagage des données

Elections municipales de 2014:

In [199]:
noms = ['Numéro tour', 'Code département', 'Code commune', 'Nom de la commune', 'Numéro de bureau de vote', 'Inscrits', 'Votants', 'Exprimés', 'Numéro de dépôt de la liste', 'Nom du candidat tête de liste', 'Prénom du candidat tête de liste', 'Code nuance de la liste', 'Nombre de voix']
municipales_t1.columns = noms
municipales_t2.columns = noms

In [222]:
def problème_FR(phrase):
    return phrase.replace('Ã©', 'é').replace('Ã«', 'ë').replace('Ã¨', 'è').replace('Ã\x89', 'É').replace('Ã´', 'ô').replace('Ã§', 'ç').replace('Ã®', 'î').replace('Ã\x8b', 'Ë').replace('Ã\x88', 'È').replace('Ã¯', 'ï')

In [223]:
cols = ['Nom de la commune', 'Nom du candidat tête de liste', 'Prénom du candidat tête de liste']
for colonne in cols:
    municipales_t1[colonne] = municipales_t1[colonne].apply(lambda row: problème_FR(row))
    municipales_t2[colonne] = municipales_t2[colonne].apply(lambda row: problème_FR(row))

On observe une singularité pour les communes de moins de 1 000 habitants.
Pour ces communes, l'élection des conseillers municipaux se déroule au scrutin majoritaire plurinominal à deux tours, avec panachage. La somme des voix de tous les candidats peut donc être supérieure au nombre de votants.

Nous allons donc ne pas pouvoir analyser ces communes.

In [246]:
res = municipales_t2.loc[municipales_t2['Votants']>1000].groupby(by=['Nom de la commune', 'Nom du candidat tête de liste'])['Nombre de voix'].sum()

In [249]:
res

Nom de la commune  Nom du candidat tête de liste
Agde               D'ETTORE                         488
                   LEBAUBE                          135
                   MUR                              433
Agny               AVRONSART                        152
                   DUTOIT                           393
                                                   ... 
Warmeriville       MOUSEL                           598
Wignehies          BAUDIN                           642
                   CESAR                            802
Wittelsheim        GOEPFERT                         506
                   RIESEMANN                        519
Name: Nombre de voix, Length: 674, dtype: int64